<a href="https://colab.research.google.com/github/arunima-22/CMT307/blob/master/part2code_%5B1655016%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#importing important modules
import numpy as np
import pandas as pd
import nltk
import sklearn
import operator
import requests
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import re
from collections import Counter
from nltk.util import ngrams
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time
from scipy.sparse import coo_matrix, hstack
from sklearn.tree import DecisionTreeClassifier
from string import punctuation
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#loading neg reviews dev
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_dev_neg.txt"
response = requests.get(url)
dev_neg = response.text.split("\n")

#loading pos reviews dev
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_dev_pos.txt"
response = requests.get(url)
dev_pos = response.text.split("\n")

#loading neg reviews test
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_test_neg.txt"
response = requests.get(url)
test_neg = response.text.split("\n")

#loading pos reviews test
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_test_pos.txt"
response = requests.get(url)
test_pos = response.text.split("\n")


#loading neg reviews train
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_train_neg.txt"
response = requests.get(url)
train_neg = response.text.split("\n")

#loading pos reviews train
url = "https://raw.githubusercontent.com/arunima-22/CMT307/master/imdb_train_pos.txt"
response = requests.get(url)
train_pos = response.text.split("\n")




In [0]:
#initiating three datasets, each having positive and negative reviews
new_dev=[]
for pos_review in dev_pos:
  new_dev.append((pos_review,1))
for neg_review in dev_neg:
  new_dev.append((neg_review,0))

  new_test=[]
for pos_review in test_pos:
  new_test.append((pos_review,1))
for neg_review in test_neg:
  new_test.append((neg_review,0))

  new_train=[]
for pos_review in train_pos:
  new_train.append((pos_review,1))
for neg_review in train_neg:
  new_train.append((neg_review,0))

In [5]:
print ("TRAINING SET")
print ("Size training set: "+str(len(new_train)))
for example in new_train[:1]:
    print (example)
print ("    \n-------\n")
print ("TEST SET")
print ("Size development set: "+str(len(new_dev)))
for example in new_dev[:1]:
  print (example)
print ("    \n-------\n")
print ("DEVELOPMENT SET")
print ("Size test set: "+str(len(new_test)))
for example in new_test[:1]:
  print (example)

TRAINING SET
Size training set: 15002
('For fans of Chris Farley, this is probably his best film. David Spade plays the perfect cynical, sarcastic yin to Farley\'s "Baby Huey" yang. Farley achieves strokes of comic genius in his monologues, like the "Let\'s say you\'re driving along the road with your family..." bit, the "Jo-Jo the Idiot Circus Boy with a pretty new pet, (his possible sale)" speech, or the "Glue-sniffing Guarantee fairy" brake pad sale. The sappy moments in the film contrast sharply with Farley and Spade\'s shenanigans. Even after many viewings, it\'s still fun to see Farley pour everything he had into the role. "Richard, what\'s HAPPENING to me?!?!"', 1)
    
-------

TEST SET
Size development set: 5002
('After 10 viewings in 20 years I too think this was the Crazy Gang\'s best effort on film, with more cohesion in the plot than their next best, "Alf\'s Button Afloat". They were indeed a crazy trio of double acts thrown together mainly on stage, sometimes in front of 

In [6]:
print ("Positive reviews:\n")
for pos_review in train_pos[:5]:
  print (pos_review)
print ("\n   ------\n")  
print ("Negative reviews:\n")
for neg_review in train_neg[:5]:
  print (neg_review)

Positive reviews:

For fans of Chris Farley, this is probably his best film. David Spade plays the perfect cynical, sarcastic yin to Farley's "Baby Huey" yang. Farley achieves strokes of comic genius in his monologues, like the "Let's say you're driving along the road with your family..." bit, the "Jo-Jo the Idiot Circus Boy with a pretty new pet, (his possible sale)" speech, or the "Glue-sniffing Guarantee fairy" brake pad sale. The sappy moments in the film contrast sharply with Farley and Spade's shenanigans. Even after many viewings, it's still fun to see Farley pour everything he had into the role. "Richard, what's HAPPENING to me?!?!"
Fantastic, Madonna at her finest, the film is funny and her acting is brilliant. It may have been made in the 80's but it has all the qualities of a modern Hollywood Block-buster. I love this film and i think its totally unique and will cheer up any droopy person within a matter of minutes. Fantastic.
From a perspective that it is possible to make m

In [7]:

lemmatizer = nltk.stem.WordNetLemmatizer()

def get_list_tokens(string):
  sentence_split=nltk.tokenize.sent_tokenize(string)
  list_tokens=[]
  for sentence in sentence_split:
    list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
    for token in list_tokens_sentence:
      list_tokens.append(lemmatizer.lemmatize(token).lower())
  return list_tokens

# First, we get the stopwords list from nltk
stopwords=set(nltk.corpus.stopwords.words('english'))
# We can add more words to the stopword list, like punctuation marks
stopwords.add(".")
stopwords.add(",")
stopwords.add("--")
stopwords.add("``")
stopwords.add("#")
stopwords.add("@")
stopwords.add(":")
stopwords.add("'s")
stopwords.add("’")
stopwords.add("...")
stopwords.add("n't")
stopwords.add("'re")
stopwords.add("'")
stopwords.add("-")
stopwords.add(";")
stopwords.add("/")
stopwords.add(">")
stopwords.add("<")
stopwords.add("br")
stopwords.add("(")
stopwords.add(")")
stopwords.add("''")
stopwords.add("&")

# Now we create a frequency dictionary with all words in the dataset


dict_word_frequency={}
for pos_review in train_pos:
  sentence_tokens=get_list_tokens(pos_review)
  for word in sentence_tokens:
    if word in stopwords: continue
    if word not in dict_word_frequency: dict_word_frequency[word]=1
    else: dict_word_frequency[word]+=1
for neg_review in train_neg:
  sentence_tokens=get_list_tokens(neg_review)
  for word in sentence_tokens:
    if word in stopwords: continue
    if word not in dict_word_frequency: dict_word_frequency[word]=1
    else: dict_word_frequency[word]+=1
      
# Now we create a sorted frequency list with the top 1000 words, using the function "sorted". Let's see the 15 most frequent words
sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:1000]
i=0
for word,frequency in sorted_list[:15]:
  i+=1
  print (str(i)+". "+word+" - "+str(frequency))
  
# Finally, we create our vocabulary based on the sorted frequency list 
vocabulary=[]
for word,frequency in sorted_list:
  vocabulary.append(word)

1. / - 59704
2. > - 59628
3. < - 59572
4. br - 59548
5. 's - 36104
6. movie - 29647
7. wa - 29577
8. film - 26929
9. ) - 21211
10. ( - 20708
11. '' - 19859
12. `` - 19693
13. n't - 19639
14. one - 15987
15. ! - 14847


In [0]:
def get_vector_text(list_vocab,string):
  vector_text=np.zeros(len(list_vocab))
  list_tokens_string=get_list_tokens(string)
  for i, word in enumerate(list_vocab):
    if word in list_tokens_string:
      vector_text[i]=list_tokens_string.count(word)
  return vector_text

In [0]:
def get_vocabulary(training_set, num_features): # Function to retrieve vocabulary
  dict_word_frequency={}
  for instance in training_set:
    sentence_tokens=get_list_tokens(instance[0])
    for word in sentence_tokens:
      if word in stopwords: continue
      if word not in dict_word_frequency: dict_word_frequency[word]=1
      else: dict_word_frequency[word]+=1
  sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:num_features]
  vocabulary=[]
  for word,frequency in sorted_list:
    vocabulary.append(word)
  return vocabulary

In [0]:
def train_svm_classifier(training_set, vocabulary): # Function for training our svm classifier
  X_train=[]
  Y_train=[]
  for instance in new_train:
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_train.append(vector_instance)
    Y_train.append(instance[1])
  # Finally, we train the SVM classifier 
  svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
  svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
  return svm_clf

In [0]:
vocabulary=get_vocabulary(new_train, 1000)

In [0]:
svm_clf=train_svm_classifier(new_train, vocabulary)

In [0]:
X_test=[]
Y_test=[]
for instance in new_test:
  vector_instance=get_vector_text(vocabulary,instance[0])
  X_test.append(vector_instance)
  Y_test.append(instance[1])
X_test=np.asarray(X_test)
Y_test_gold=np.asarray(Y_test)

In [0]:
from sklearn.metrics import classification_report

In [0]:
Y_text_predictions=svm_clf.predict(X_test)

In [0]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

In [17]:
precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.848
Recall: 0.848
F1-Score: 0.848
Accuracy: 0.848


In [19]:

Y_dev=[]
for instance in new_dev:
  Y_dev.append(instance[1])
Y_dev_gold=np.asarray(Y_dev)

# Now we can train our three models with the different number of features, and test each of them in the dev set
list_num_features=[250,500,750,1000]
best_accuracy_dev=0.0
for num_features in list_num_features:
  # First, we get the vocabulary from the training set and train our svm classifier
  vocabulary=get_vocabulary(new_train, num_features)  
  svm_clf=train_svm_classifier(new_train, vocabulary)
  # Then, we transform our dev set into vectors and make the prediction on this set
  X_dev=[]
  for instance in new_dev:
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_dev.append(vector_instance)
  X_dev=np.asarray(X_dev)
  Y_dev_predictions=svm_clf.predict(X_dev)
  # Finally, we get the accuracy results of the classifier
  accuracy_dev=accuracy_score(Y_dev_gold, Y_dev_predictions)
  print ("Accuracy with "+str(num_features)+": "+str(round(accuracy_dev,3)))
  if accuracy_dev>=best_accuracy_dev:
    best_accuracy_dev=accuracy_dev
    best_num_features=num_features
    best_vocabulary=vocabulary
    best_svm_clf=svm_clf
print ("\n Best accuracy overall in the dev set is "+str(round(best_accuracy_dev,3))+" with "+str(best_num_features)+" features.")

Accuracy with 250: 0.783
Accuracy with 500: 0.834
Accuracy with 750: 0.845
Accuracy with 1000: 0.846

 Best accuracy overall in the dev set is 0.846 with 1000 features.


In [30]:
vectorizer = TfidfVectorizer(ngram_range=(1,3),max_features=2000)
matrix = vectorizer.fit_transform(np.asarray([i[0] for i in new_train]))
X_train = matrix.toarray()
Y_train=[i[1] for i in new_train]

svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))

X_dev = vectorizer.transform([i[0] for i in new_dev]).toarray()
predictions = svm_clf.predict(X_dev)
print(sklearn.metrics.classification_report(predictions,[i[1] for i in new_dev]))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2427
           1       0.88      0.86      0.87      2575

    accuracy                           0.87      5002
   macro avg       0.87      0.87      0.87      5002
weighted avg       0.87      0.87      0.87      5002



In [0]:
#the chi-squared test method is used. This method basically removes the features that appear to be irrelevant to a given class (in our case positive or negative)
vectorizer_f = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
X_train = vectorizer_f.fit_transform([i[0] for i in new_train]).toarray()
Y_train = np.asarray([i[1] for i in new_train])

X_feature_best = SelectKBest(chi2, k=500).fit(X_train, Y_train)
X_train_feature_best = X_feature_best.transform(X_train)

X_test = vectorizer_f.transform([i[0] for i in new_test]).toarray()
X_test_feature_best =  X_feature_best.transform(X_test)

Y_test = np.asarray([i[1] for i in new_test])

In [48]:
#final accuracy, recall, precision and f-measure.
new_svm_clf_sentanalysis_=sklearn.svm.SVC(kernel="linear",gamma='auto')
new_svm_clf_sentanalysis_.fit(np.asarray(X_train_feature_best),Y_train)
predictions_feature_best = new_svm_clf_sentanalysis_.predict(X_test_feature_best)
print(sklearn.metrics.classification_report(predictions_feature_best,Y_test))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      2376
           1       0.89      0.85      0.87      2626

    accuracy                           0.87      5002
   macro avg       0.87      0.87      0.87      5002
weighted avg       0.87      0.87      0.87      5002

